Syntax natürlicher Sprachen, WS 2023/24

# 10 - Aufgabenblatt (Lösung)

In [1]:
import nltk

## Aufgabe 1 - Gewichte und Ableitungswahrscheinlichkeit

#### Betrachten Sie folgende (unvollständige) PCFG:

In [ ]:
grammar = nltk.PCFG.fromstring("""
    S -> NP VP [**]
    NP -> PROPN [**]
    VP -> V NP [0.3]
    VP -> V NP ADJP [**]
    ADJP -> ADJ [**]
    ADJP -> PRT ADJP [0.1]

    PROPN -> "Maria" [0.5]
    PROPN -> "Moritz" [**]
    V -> "kennt" [**]
    PRT -> "sehr" [**]
    ADJ -> "gut" [**]
""")

sent = "Maria kennt Moritz sehr sehr gut"

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(sent.split()):
    tree.pretty_print(unicodelines=True)
    print(tree)

#### Ergänzen Sie die fehlenden (mit ** markierten) Gewichte und geben Sie die Berechnung für die Ableitungswahrscheinlichkeit durch den Viterbi-Parser an.

In [2]:
grammar = nltk.PCFG.fromstring("""
    S -> NP VP [1]
    NP -> PROPN [1]
    VP -> V NP [0.3]
    VP -> V NP ADJP [0.7]
    ADJP -> ADJ [0.9]
    ADJP -> PRT ADJP [0.1]

    PROPN -> "Maria" [0.5]
    PROPN -> "Moritz" [0.5]
    V -> "kennt" [1]
    PRT -> "sehr" [1]
    ADJ -> "gut" [1]
""")

sent = "Maria kennt Moritz sehr sehr gut"

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(sent.split()):
    tree.pretty_print(unicodelines=True)
    print(tree)

              S                       
  ┌───────────┴─────┐                  
  │                 VP                
  │     ┌─────┬─────┴────┐             
  │     │     │         ADJP          
  │     │     │     ┌────┴────┐        
  │     │     │     │        ADJP     
  │     │     │     │    ┌────┴────┐   
  NP    │     NP    │    │        ADJP
  │     │     │     │    │         │   
PROPN   V   PROPN  PRT  PRT       ADJ 
  │     │     │     │    │         │   
Maria kennt Moritz sehr sehr      gut 

(S
  (NP (PROPN Maria))
  (VP
    (V kennt)
    (NP (PROPN Moritz))
    (ADJP (PRT sehr) (ADJP (PRT sehr) (ADJP (ADJ gut)))))) (p=0.001575)


In [ ]:
# Berechnung der Ableitungswahrscheinlichkeit:
# 0.001575 (1*1*0.7*1* 0.1*0.1*0.9 * 0.5*1*0.5*1*1*1)

## Aufgabe 2  - PCFG Parsing
#### Betrachten Sie das folgende PCFG-Parsing:

In [2]:
grammar = nltk.PCFG.fromstring('''
  NP  -> NNS [0.5] | JJ NNS [0.3] | NP CC NP [0.2]
  NNS -> "cats" [0.1] | "dogs" [0.2] | "mice" [0.3] | NNS CC NNS [0.4]
  JJ  -> "big" [0.4] | "small" [0.6]
  CC  -> "and" [0.9] | "or" [0.1]
  ''')

sent = "big cats and dogs"

viterbi_parser = nltk.ViterbiParser(grammar)
for tree in viterbi_parser.parse(sent.split()):
    print(tree)
    tree.pretty_print(unicodelines=True)


(NP (JJ big) (NNS (NNS cats) (CC and) (NNS dogs))) (p=0.000864)
          NP         
 ┌────────┴───┐       
 │           NNS     
 │   ┌────────┼───┐   
 JJ NNS       CC NNS 
 │   │        │   │   
big cats     and dogs



### a) Berechnen Sie die Wahrscheinlichkeit für die Ableitung mit dem Viterbi-Parser.

In [ ]:
# Berechnung der Ableitungswahrscheinlichkeit:
0.000864 (0.3*0.4*0.4*0.1*0.9*0.2)

### b) Betrachten Sie das dazugehörige Tracing-Output des ViterbiParsers. Warum werden die Analysen in den Discard-Zeilen verworfen?

```
>>> sent = 'big cats and dogs'.split()
>>> viterbi_parser = nltk.ViterbiParser(grammar, trace = 3)
>>> for tree in viterbi_parser.parse(sent):
>>>     print(tree)

Inserting tokens into the most likely constituents table...
   Insert: |=...| big
   Insert: |.=..| cats
   Insert: |..=.| and
   Insert: |...=| dogs
Finding the most likely constituents spanning 1 text elements...
   Insert: |=...| JJ -> 'big' [0.4]                 0.4000000000
   Insert: |.=..| NNS -> 'cats' [0.1]               0.1000000000
   Insert: |.=..| NP -> NNS [0.5]                   0.0500000000
   Insert: |..=.| CC -> 'and' [0.9]                 0.9000000000
   Insert: |...=| NNS -> 'dogs' [0.2]               0.2000000000
   Insert: |...=| NP -> NNS [0.5]                   0.1000000000
Finding the most likely constituents spanning 2 text elements...
   Insert: |==..| NP -> JJ NNS [0.3]                0.0120000000
Finding the most likely constituents spanning 3 text elements...
   Insert: |.===| NP -> NP CC NP [0.2]              0.0009000000
   Insert: |.===| NNS -> NNS CC NNS [0.4]           0.0072000000
   Insert: |.===| NP -> NNS [0.5]                   0.0036000000
  Discard: |.===| NP -> NP CC NP [0.2]              0.0009000000
  Discard: |.===| NP -> NP CC NP [0.2]              0.0009000000
Finding the most likely constituents spanning 4 text elements...
   Insert: |====| NP -> JJ NNS [0.3]                0.0008640000
  Discard: |====| NP -> NP CC NP [0.2]              0.0002160000
  Discard: |====| NP -> NP CC NP [0.2]              0.0002160000
(NP (JJ big) (NNS (NNS cats) (CC and) (NNS dogs))) (p=***)
```

In [ ]:
# Antwort:
# Ihre Wahrscheinlichkeiten sind zu gering.

### c) Ein statistischer ChartParser findet folgende 2 Ableitungen für die NP big cats and dogs. Um welchen probabilistischen Parsingalgorithmus kann es sich NICHT handeln?
```
(NP (JJ big) (NNS (NNS cats) (CC and) (NNS dogs))) (p=0.000864)
(NP (NP (JJ big) (NNS cats)) (CC and) (NP (NNS dogs))) (p=0.000216)
```

In [3]:
from nltk import Tree
tree1 = Tree.fromstring("(NP (JJ big) (NNS (NNS cats) (CC and) (NNS dogs)))")
tree2 = Tree.fromstring("(NP (NP (JJ big) (NNS cats)) (CC and) (NP (NNS dogs)))")
tree1.pretty_print(unicodelines=True)
tree2.pretty_print(unicodelines=True)

          NP         
 ┌────────┴───┐       
 │           NNS     
 │   ┌────────┼───┐   
 JJ NNS       CC NNS 
 │   │        │   │   
big cats     and dogs

         NP          
     ┌───┴────┬───┐   
     NP       │   NP 
 ┌───┴───┐    │   │   
 JJ     NNS   CC NNS 
 │       │    │   │   
big     cats and dogs



In [ ]:
# Antwort:
# ViterbiParser (da mehr als 1 Ableitung, kann z.B. InsideChartParser (mit/ohne beam size) oder LongestChartParser sein)

### d) Nach welchem Kriterium wird beim Parsen mit dem InsideChartParser (= Lowest-Cost-First-Strategie) die *edge queue* sortiert?

In [ ]:
# Antwort:
# nach der Wahrscheinlichkeit
# (nicht nach der Länge der Ableitung oder beam size)